## D4. Dataset filtering and sampling

#### 0. Load required libraries, site dependant constants and utility functions.

In [1]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import seaborn as sns
import datetime
from os import listdir
from os.path import isfile, join

# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission
from eolearn.features.feature_manipulation import SimpleFilterTask
from eolearn.geometry.sampling import PointSamplingTask

# Add to python path parent dictionary
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

C:\Users\Kamil\Anaconda3\lib\site-packages\pywt\_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


#### 1. Define  EOTASKs

In [2]:
""" Predicate that defines if a frame from EOPatch's time-series is valid or not.
Frame is valid, if the valid mask fraction is below the specified threshold."""    

class InvalidDataFractionPredicate:
    def __init__(self, threshold):
        self.threshold = threshold
        
    def __call__(self, array):
        mask_coverage = np.sum(array.astype(np.uint8)) / np.prod(array.shape)
        return mask_coverage < self.threshold
    
""" Predicate that defines if a frame from EOPatch's time-series is valid or not.
Frame is valid, if minimum extracted water feature exceeds the specified threshold."""    

class MinimumWaterFeaturePredicate:
    def __init__(self, minimum_water_feature_count = 139 ): # 139 bc it is same what 0,125 km2
        self.minimum_water_feature_count = minimum_water_feature_count
        
    def __call__(self, array):
        cur_patch_water_feature_count = np.sum(array.astype(np.uint8))
        return self.minimum_water_feature_count <= cur_patch_water_feature_count    


#### History:
1. min wat - 5 , sam 100 ref 1,0- 8% of water
2. min wat- 5 , sam 200 ref 1,0- 8% of water
3. min wat- 5 , sam 20000 ref 1,0- 8% of water
4. min wat- 25 , sam 20000 ref 1,0- 9,7% of water
5. min wat- 25 , sam 20k, ref 1 - 20, % of water features
6. min wat- 50 , sam 80k, ref 0, 1 - real distribution d
7. min wat- 25 , sam 25k, ref 1, 2 - 

In [23]:
list(range(1,3))

[1, 2]

In [32]:
# Defire filter eo_tasks,
min_wat_threshold = 25
max_cld_threshold = 0.01
dem_std_threshold = 125
n_samples = 25000 # no. of pixels to sample   50 000 / 110 224 = 0.45362171577 , every second pixel will be taken into account
dataset_file_name = '_sptl_smpl_25k_pr_pic_ref_1_2_min_wat_25'
# Define what kind of feature are going to be used:
ref_labels = list(range(1,3)) # reference labels to take into account when sampling - 
                            # -1 - never to take - frame mask,  0 - never considered as water, 1, considered once or more

invalid_data_predicate = InvalidDataFractionPredicate(max_cld_threshold)
filter_ultimate_mask_eotask = SimpleFilterTask((FeatureType.MASK, 'INVALID_DATA_MASK_CLD_150_STR2-IS_LDS-IS_MDS'), invalid_data_predicate)

minimum_water_predicate  = MinimumWaterFeaturePredicate(min_wat_threshold)
filter_by_minimum_water_count_eotask = SimpleFilterTask((FeatureType.MASK, 'WATER_MASK_ST_025'), minimum_water_predicate)

In [33]:
eo_patch = io_utils.load_exemplary_eopatch(date_range = site.DATE_RANGES_LANDSAT_8[0], patch_id=6, random_choice=True)
eo_patch

Loaded from ../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2013_05-2013_10/2976_maxcc_0.05_x-3_y-137


EOPatch(
  data: {
    DATASET_CLD_200_dil_6_str2: numpy.ndarray(shape=(7, 333, 333, 11), dtype=float32)
    DATASET_RAW_NDWI_NRB_19_NORM_SLOPE: numpy.ndarray(shape=(7, 333, 333, 13), dtype=float32)
    LANDSAT_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_GREEN_NIR: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_ICE: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_1: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_2: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_QA_LAYERS: numpy.ndarray(shape=(7, 333, 333, 5), dtype=float32)
    LANDSAT_RAW_BANDS: numpy.ndarray(shape=(7, 333, 333, 11), dtype=float32)
    MODIS_AVG_NBR_19: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    MODIS_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    MODIS_NDWI

In [34]:
eo_patch.scalar_timeless['WATER_MASK_ST_025_COVERAGE']

array([  0,   0,   0,  38,   0, 104,   0], dtype=uint32)

In [35]:
filter_by_minimum_water_count_eotask.execute(eo_patch)

EOPatch(
  data: {
    DATASET_CLD_200_dil_6_str2: numpy.ndarray(shape=(2, 333, 333, 11), dtype=float32)
    DATASET_RAW_NDWI_NRB_19_NORM_SLOPE: numpy.ndarray(shape=(2, 333, 333, 13), dtype=float32)
    LANDSAT_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_GREEN_NIR: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_ICE: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_1: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_2: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    LANDSAT_QA_LAYERS: numpy.ndarray(shape=(2, 333, 333, 5), dtype=float32)
    LANDSAT_RAW_BANDS: numpy.ndarray(shape=(2, 333, 333, 11), dtype=float32)
    MODIS_AVG_NBR_19: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    MODIS_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(2, 333, 333, 1), dtype=float32)
    MODIS_NDWI

In [36]:
ref_labels

[1, 2]

In [37]:
# TASK FOR SPATIAL SAMPLING
# Uniformly sample about pixels from patches


spatial_sampling_eotask = PointSamplingTask(
    n_samples=n_samples, 
    ref_mask_feature='WATER_MASK_ST_025_FR_1_MAPPED_2', 
    ref_labels=ref_labels,
    even_sampling=False,
    return_new_eopatch=True,
    sample_features=[  # tag fields to sample
        (FeatureType.DATA, 'DATASET_RAW_NDWI_NRB_19_NORM_SLOPE'),
        (FeatureType.MASK, 'WATER_MASK_ST_025')
    ])

In [20]:
site.DATE_RANGES_LANDSAT_8[3:]

[('2016-05-01', '2016-10-31'),
 ('2017-05-01', '2017-10-31'),
 ('2018-05-01', '2018-10-31'),
 ('2019-05-01', '2019-10-31')]

In [38]:
%%time

for date_range in site.DATE_RANGES_LANDSAT_8:
    print('Processing EOPatches from:', date_range )
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[1], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range)
    print('Reading EOPatches from:', eopatches_filepath )
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    
    path_out_sampled = eopatches_filepath[:-1]+ dataset_file_name
    if not os.path.isdir(path_out_sampled):
        os.makedirs(path_out_sampled)
    save_eotask = SaveTask(path_out_sampled, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    #save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    filter_ultimate_mask_eotask,
    filter_by_minimum_water_count_eotask,
    spatial_sampling_eotask,
    save_eotask
    )
    
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    list_in_chunks = io_utils.chunkIt(list_of_available_patches, 5 )
    for chunk in list_in_chunks:
        print('Doing now following eopatches:', chunk )
        execution_args = []
        for eopatch_name in chunk:
            try: 
                eo_patch_test = EOPatch.load(eopatches_filepath+eopatch_name)
            except OSError as err:
                print("OS error: {0}".format(err)) 
                print("while trying to load:  {0}".format(eopatches_filepath+eopatch_name)) 
                continue
            # eo_patch based filter to get rif of eopatches with high variance in digital elevation model
            # first we would have to calculation hillshade to exlcude the shades from te pictures
            # threshold 125 was selected based on the distribution of std
            if eo_patch_test.meta_info['DEM_RAW_STD'] > dem_std_threshold:
                continue
                
            execution_args.append({
            load_eotask: {'eopatch_folder': eopatch_name},
            save_eotask: {'eopatch_folder': eopatch_name}
            })
                                     
        executor = EOExecutor(workflow, execution_args, save_logs=True)
        executor.run(workers=None, multiprocess=False)
        executor.make_report()

Processing EOPatches from: ('2013-05-01', '2013-10-31')
Reading EOPatches from: ../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2013_05-2013_10/
Doing now following eopatches: ['2969_maxcc_0.05_x-3_y-130', '2970_maxcc_0.05_x-3_y-131', '2971_maxcc_0.05_x-3_y-132', '2972_maxcc_0.05_x-3_y-133', '2973_maxcc_0.05_x-3_y-134', '2974_maxcc_0.05_x-3_y-135', '2975_maxcc_0.05_x-3_y-136', '2976_maxcc_0.05_x-3_y-137', '2977_maxcc_0.05_x-3_y-138', '2978_maxcc_0.05_x-3_y-139', '2979_maxcc_0.05_x-3_y-140']



Doing now following eopatches: ['3059_maxcc_0.05_x-4_y-130', '3060_maxcc_0.05_x-4_y-131', '3061_maxcc_0.05_x-4_y-132', '3062_maxcc_0.05_x-4_y-133', '3063_maxcc_0.05_x-4_y-134', '3064_maxcc_0.05_x-4_y-135', '3065_maxcc_0.05_x-4_y-136', '3066_maxcc_0.05_x-4_y-137', '3067_maxcc_0.05_x-4_y-138', '3068_maxcc_0.05_x-4_y-139', '3069_maxcc_0.05_x-4_y-140']



Doing now following eopatches: ['3172_maxcc_0.05_x-5_y-130', '3173_maxcc_0.05_x-5_y-131', '3174_maxcc_0.05_x-5_y-132', '3175_maxcc_0.05_x-5_y-133', '3176_maxcc_0.05_x-5_y-134', '3177_maxcc_0.05_x-5_y-135', '3178_maxcc_0.05_x-5_y-136', '3179_maxcc_0.05_x-5_y-137', '3180_maxcc_0.05_x-5_y-138', '3181_maxcc_0.05_x-5_y-139', '3182_maxcc_0.05_x-5_y-140']



Doing now following eopatches: ['3306_maxcc_0.05_x-6_y-130', '3307_maxcc_0.05_x-6_y-131', '3308_maxcc_0.05_x-6_y-132', '3309_maxcc_0.05_x-6_y-133', '3310_maxcc_0.05_x-6_y-134', '3311_maxcc_0.05_x-6_y-135', '3312_maxcc_0.05_x-6_y-136', '3313_maxcc_0.05_x-6_y-137', '3314_maxcc_0.05_x-6_y-138', '3315_maxcc_0.05_x-6_y-139', '3316_maxcc_0.05_x-6_y-140']



Doing now following eopatches: ['3469_maxcc_0.05_x-7_y-130', '3470_maxcc_0.05_x-7_y-131', '3471_maxcc_0.05_x-7_y-132', '3472_maxcc_0.05_x-7_y-133', '3473_maxcc_0.05_x-7_y-134', '3474_maxcc_0.05_x-7_y-135', '3475_maxcc_0.05_x-7_y-136', '3476_maxcc_0.05_x-7_y-137', '3477_maxcc_0.05_x-7_y-138', '3478_maxcc_0.05_x-7_y-139', '3479_maxcc_0.05_x-7_y-140']


...


Doing now following eopatches: ['3059_maxcc_0.05_x-4_y-130', '3060_maxcc_0.05_x-4_y-131', '3061_maxcc_0.05_x-4_y-132', '3062_maxcc_0.05_x-4_y-133', '3063_maxcc_0.05_x-4_y-134', '3064_maxcc_0.05_x-4_y-135', '3065_maxcc_0.05_x-4_y-136', '3066_maxcc_0.05_x-4_y-137', '3067_maxcc_0.05_x-4_y-138', '3068_maxcc_0.05_x-4_y-139', '3069_maxcc_0.05_x-4_y-140']



Doing now following eopatches: ['3172_maxcc_0.05_x-5_y-130', '3173_maxcc_0.05_x-5_y-131', '3174_maxcc_0.05_x-5_y-132', '3175_maxcc_0.05_x-5_y-133', '3176_maxcc_0.05_x-5_y-134', '3177_maxcc_0.05_x-5_y-135', '3178_maxcc_0.05_x-5_y-136', '3179_maxcc_0.05_x-5_y-137', '3180_maxcc_0.05_x-5_y-138', '3181_maxcc_0.05_x-5_y-139', '3182_maxcc_0.05_x-5_y-140']



Doing now following eopatches: ['3306_maxcc_0.05_x-6_y-130', '3307_maxcc_0.05_x-6_y-131', '3308_maxcc_0.05_x-6_y-132', '3309_maxcc_0.05_x-6_y-133', '3310_maxcc_0.05_x-6_y-134', '3311_maxcc_0.05_x-6_y-135', '3312_maxcc_0.05_x-6_y-136', '3313_maxcc_0.05_x-6_y-137', '3314_maxcc_0.05_x-6_y-138', '3315_maxcc_0.05_x-6_y-139', '3316_maxcc_0.05_x-6_y-140']



Doing now following eopatches: ['3469_maxcc_0.05_x-7_y-130', '3470_maxcc_0.05_x-7_y-131', '3471_maxcc_0.05_x-7_y-132', '3472_maxcc_0.05_x-7_y-133', '3473_maxcc_0.05_x-7_y-134', '3474_maxcc_0.05_x-7_y-135', '3475_maxcc_0.05_x-7_y-136', '3476_maxcc_0.05_x-7_y-137', '3477_maxcc_0.05_x-7_y-138', '3478_maxcc_0.05_x-7_y-139', '3479_maxcc_0.05_x-7_y-140']



Wall time: 49min 3s


#### 4. Inspect data
load eopatch and check how does it look like

In [18]:
file_patch = path_out_sampled
print(file_patch)

exemplary_landsat_1 = EOPatch.load(file_patch+'3469_maxcc_0.05_x-7_y-130')
exemplary_landsat_1

../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2019_05-2019_10_sptl_smpl_20000_pr_patch_blanced_only_wat_1_min_wat_50/


EOPatch(
  data: {
    DATASET_CLD_200_dil_6_str2_SAMPLED: numpy.ndarray(shape=(1, 20000, 1, 11), dtype=float32)
  }
  mask: {
    WATER_MASK_ST_025_SAMPLED: numpy.ndarray(shape=(1, 20000, 1, 1), dtype=bool)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {
    DEM_RAW_AVG: 1668.2764
    DEM_RAW_MAX: 1792.0
    DEM_RAW_MED: 1670.0
    DEM_RAW_MIN: 1461.0
    DEM_RAW_STD: 74.915634
    DEM_RAW_VAR: 5612.3525
    index_x: 7
    index_y: 130
    maxcc: '0.05'
    patch_index: 3469
    service_type: 'wcs'
    site_name: 'UPE_PROMICE'
    size_x: '30m'
    size_y: '30m'
    time_difference: datetime.timedelta(seconds=7200)
    time_interval: ('2019-05-01', '2019-10-31')
  }
  bbox: BBox(((440000.0, 8030000.0), (450000.0, 8040000.0)), crs=EPSG:32622)
  timestamp: [datetime.datetime(2019, 8, 3, 15, 42, 44)]
)

In [40]:
print(path_out_sampled )

../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2019_05-2019_10_sptl_smpl_25k_pr_pic_ref_1_2_min_wat_25
